## Transform Orders Data - Explode Arrays
1. Access elements from the JSON object
1. Deduplicate Array Elements
1. Explode Arrays
1. Write the Transformed Data to Silver Schema

In [0]:

df_orders = spark.table('gizmobox.silver.py_orders_json')
display(df_orders)

In [0]:
# 1. 

from pyspark.sql import functions as F

df_orders_normalized = (
    df_orders.select(
        'json_value.order_id',
        'json_value.order_status',
        'json_value.payment_method',
        'json_value.total_amount',
        'json_value.transaction_timestamp',
        'json_value.customer_id',
        'json_value.items'
    )
)
display(df_orders_normalized)

In [0]:
# 2. 

from pyspark.sql import functions as F

df_orders_normalized = (
    df_orders.select(
        'json_value.order_id',
        'json_value.order_status',
        'json_value.payment_method',
        'json_value.total_amount',
        'json_value.transaction_timestamp',
        'json_value.customer_id',
        F.array_distinct('json_value.items').alias('items')
    )
)
display(df_orders_normalized)

In [0]:
# 3.
df_orders_exploded = (
    df_orders_normalized
    .select(
        "order_id",
        "order_status",
        "payment_method",
        "total_amount",
        "transaction_timestamp",
        "customer_id",
        F.explode("items").alias("item")
    )
)

display(df_orders_exploded)

In [0]:
df_order_items = (
    df_orders_exploded.select(
        "order_id",
        "order_status",
        "payment_method",
        "total_amount",
        "transaction_timestamp",
        "customer_id",
        "item.item_id",
        "item.name",
        "item.price",
        "item.category",
        "item.details.brand",
        "item.details.color", 
        "item.quantity" 
         )
)
display(df_order_items)

In [0]:
# 4.

df_order_items.writeTo('gizmobox.silver.py_orders').createOrReplace()

In [0]:
%sql
select * from gizmobox.silver.py_orders